In [ ]:
#instalar libreria optuna
!pip install optuna

El objetivo del programa es encontrar los mejores parámetros para un modelo de Random Forest. Para lograrlo usamos Optuna, que se encarga de explorar y seleccionar las combinaciones que ofrecen el mejor rendimiento.

Optuna usa una optimización estocástica, donde los hiperparámetros se van probando de forma aleatoria o adaptativa según lo que va aprendiendo. Además, emplea optimización bayesiana para modelar cómo se comporta la función objetivo y así dirigir la búsqueda hacia las zonas donde es más probable obtener mejores resultados. A la vez, va descartando los trials que rinden mal, para no seguir perdiendo tiempo en configuraciones poco prometedoras.

In [2]:
# codigo para descubrir los mejores hiperparametros
import pandas as pd
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
import optuna.visualization as vis
import plotly.graph_objects as go

# cargar datos
df = pd.read_csv('dataset.csv')
X = df.drop(['id','target_variable'], axis=1)
y = df['target_variable']

# rangos de busqueda de los hiperparámetros
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 500)
    max_depth = trial.suggest_int('max_depth', 5, 50)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 5, 10)  # mínimo 5 para evitar overfitting
    max_features = trial.suggest_categorical('max_features', ['sqrt','log2', None])

    rf = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        class_weight='balanced',
        random_state=42,
        n_jobs=-1  # usamos todos los núcleos
    )

    # 3 folds del cross validation
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    score = cross_val_score(rf, X, y, cv=cv, scoring='accuracy', n_jobs=-1).mean()
    return score

# crear y ejecutar el estudio
study = optuna.create_study(direction='maximize')  # se busca maximizar la acuracy
study.optimize(objective, n_trials=30)  # 30 evaluaciones de hiperparámetros

# filtrar y mostrar solo los resultados con score > 0.8
best_trials = [trial for trial in study.trials if trial.value is not None and trial.value > 0.8]

if best_trials: # comprobar si la lista best_trials no está vacía.

    # ordenar best_trials por su value de mayor a menor.
    best_trials.sort(key=lambda x: x.value, reverse=True)

    print("Trials con accuracy > 0.8:")
    for i, trial in enumerate(best_trials[:3]):  # Mostrar top 3
        print(f"\nTrial #{i+1}:")
        print(f"Score: {trial.value:.4f}")
        print(f"Hiperparámetros: {trial.params}")

    print(f"\nTotal de trials con accuracy > 0.8: {len(best_trials)}")
    print(f"\nMejores hiperparámetros: {study.best_params}")
    print(f"Mejor score CV: {study.best_value:.4f}")

else:
    print("No se encontraron trials con accuracy mayor a 0.8")
    print(f"Mejor score encontrado: {study.best_value:.4f}")


# GRAFICAS

# 1. Parallel Coordinate
fig1 = vis.plot_parallel_coordinate(study)
fig1.update_layout(title="1. Parallel Coordinate: Influencia de Hiperparámetros")
fig1.show()

# 2. Importance Plot
fig2 = vis.plot_param_importances(study)
fig2.update_layout(title="2. Importancia de los Hiperparámetros")
fig2.show()



[I 2025-11-19 14:47:49,642] A new study created in memory with name: no-name-3f5a34e0-c0bc-4834-8616-fba0f2a8625f
[I 2025-11-19 14:48:07,536] Trial 0 finished with value: 0.7465109599060238 and parameters: {'n_estimators': 173, 'max_depth': 22, 'min_samples_split': 14, 'min_samples_leaf': 9, 'max_features': 'log2'}. Best is trial 0 with value: 0.7465109599060238.
[I 2025-11-19 14:48:33,877] Trial 1 finished with value: 0.7974593884179693 and parameters: {'n_estimators': 113, 'max_depth': 33, 'min_samples_split': 6, 'min_samples_leaf': 8, 'max_features': None}. Best is trial 1 with value: 0.7974593884179693.
[I 2025-11-19 14:48:38,931] Trial 2 finished with value: 0.6787374911999512 and parameters: {'n_estimators': 117, 'max_depth': 9, 'min_samples_split': 7, 'min_samples_leaf': 8, 'max_features': 'log2'}. Best is trial 1 with value: 0.7974593884179693.
[I 2025-11-19 14:48:54,149] Trial 3 finished with value: 0.7679042879088032 and parameters: {'n_estimators': 265, 'max_depth': 32, 'min

Trials con accuracy > 0.8:

Trial #1:
Score: 0.8154
Hiperparámetros: {'n_estimators': 407, 'max_depth': 50, 'min_samples_split': 5, 'min_samples_leaf': 5, 'max_features': None}

Trial #2:
Score: 0.8152
Hiperparámetros: {'n_estimators': 338, 'max_depth': 29, 'min_samples_split': 3, 'min_samples_leaf': 5, 'max_features': None}

Trial #3:
Score: 0.8144
Hiperparámetros: {'n_estimators': 321, 'max_depth': 38, 'min_samples_split': 2, 'min_samples_leaf': 5, 'max_features': None}

Total de trials con accuracy > 0.8: 15

Mejores hiperparámetros: {'n_estimators': 407, 'max_depth': 50, 'min_samples_split': 5, 'min_samples_leaf': 5, 'max_features': None}
Mejor score CV: 0.8154


### Conclusiones

- max_depth es el hiperparámetro más importante.
- max_features es el segundo más importante.
- max_features = None genera casi todos los mejores resultados (>0.81).
- El max_depth óptimo está entre 25 y 35.
- min_samples_leaf = 5 aparece en todos los mejores modelos.
- n_estimators entre 180 y 230 funciona igual que usar 400–500, por lo que tiene poca importancia.
- La accuracy no mejora usando sqrt o log2 como max_features.

---

### Acciones a tomar

- Ajustar los rangos de los hiperparámetros más importantes.
- Reducir max_depth al rango 20–40.
- Fijar max_features = None.
- Fijar min_samples_leaf = 5.
- Reducir el rango de n_estimators, ya que entre 180–240 funciona igual que valores más altos.
- Afinar min_samples_split alrededor de 8–9.


In [3]:
# codigo para descubrir los mejores hiperparametros optimizado por optuna
import pandas as pd
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
import optuna.visualization as vis
import plotly.graph_objects as go

# cargar datos
df = pd.read_csv('dataset.csv')
X = df.drop(['id','target_variable'], axis=1)
y = df['target_variable']

# rangos de busqueda de los hiperparámetros
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 150, 300)         # modificado
    max_depth = trial.suggest_int('max_depth', 20, 40)                 # modificado
    min_samples_split = trial.suggest_int('min_samples_split', 5, 12)  # modificado
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 5, 10)    # mínimo 5 para evitar overfitting
    max_features = trial.suggest_categorical('max_features', ['sqrt','log2', None])

    rf = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=5,         #fijado
        max_features=None,          #fijado
        class_weight='balanced',
        random_state=42,
        n_jobs=-1  # usamos todos los núcleos
    )

    # 3 folds del cross validation
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    score = cross_val_score(rf, X, y, cv=cv, scoring='accuracy', n_jobs=-1).mean()
    return score

# crear y ejecutar el estudio
study = optuna.create_study(direction='maximize')  # se busca maximizar la acuracy
study.optimize(objective, n_trials=30)  # 30 evaluaciones de hiperparámetros

# filtrar y mostrar solo los resultados con score > 0.8
best_trials = [trial for trial in study.trials if trial.value is not None and trial.value > 0.8]

if best_trials: # comprobar si la lista best_trials no está vacía.

    # ordenar best_trials por su value de mayor a menor.
    best_trials.sort(key=lambda x: x.value, reverse=True)

    print("Trials con accuracy > 0.8:")
    for i, trial in enumerate(best_trials[:3]):  # Mostrar top 3
        print(f"\nTrial #{i+1}:")
        print(f"Score: {trial.value:.4f}")
        print(f"Hiperparámetros: {trial.params}")

    print(f"\nTotal de trials con accuracy > 0.8: {len(best_trials)}")
    print(f"\nMejores hiperparámetros: {study.best_params}")
    print(f"Mejor score CV: {study.best_value:.4f}")

else:
    print("No se encontraron trials con accuracy mayor a 0.8")
    print(f"Mejor score encontrado: {study.best_value:.4f}")


# GRAFICAS

# 1. Parallel Coordinate
fig1 = vis.plot_parallel_coordinate(study)
fig1.update_layout(title="1. Parallel Coordinate: Influencia de Hiperparámetros")
fig1.show()

# 2. Importance Plot
fig2 = vis.plot_param_importances(study)
fig2.update_layout(title="2. Importancia de los Hiperparámetros")
fig2.show()



[I 2025-11-19 15:12:25,495] A new study created in memory with name: no-name-6bdbcf8f-054d-4492-85ad-52bd378f5aee
[I 2025-11-19 15:13:07,720] Trial 0 finished with value: 0.8138386980524185 and parameters: {'n_estimators': 166, 'max_depth': 39, 'min_samples_split': 9, 'min_samples_leaf': 7, 'max_features': 'log2'}. Best is trial 0 with value: 0.8138386980524185.
[I 2025-11-19 15:13:56,704] Trial 1 finished with value: 0.8137550976757759 and parameters: {'n_estimators': 243, 'max_depth': 31, 'min_samples_split': 11, 'min_samples_leaf': 5, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.8138386980524185.
[I 2025-11-19 15:14:34,577] Trial 2 finished with value: 0.8140615214319741 and parameters: {'n_estimators': 179, 'max_depth': 35, 'min_samples_split': 8, 'min_samples_leaf': 6, 'max_features': 'sqrt'}. Best is trial 2 with value: 0.8140615214319741.
[I 2025-11-19 15:15:30,252] Trial 3 finished with value: 0.8140058150051367 and parameters: {'n_estimators': 274, 'max_depth': 38, '

Trials con accuracy > 0.8:

Trial #1:
Score: 0.8147
Hiperparámetros: {'n_estimators': 210, 'max_depth': 36, 'min_samples_split': 6, 'min_samples_leaf': 6, 'max_features': None}

Trial #2:
Score: 0.8146
Hiperparámetros: {'n_estimators': 193, 'max_depth': 36, 'min_samples_split': 6, 'min_samples_leaf': 7, 'max_features': None}

Trial #3:
Score: 0.8146
Hiperparámetros: {'n_estimators': 209, 'max_depth': 37, 'min_samples_split': 6, 'min_samples_leaf': 6, 'max_features': None}

Total de trials con accuracy > 0.8: 30

Mejores hiperparámetros: {'n_estimators': 210, 'max_depth': 36, 'min_samples_split': 6, 'min_samples_leaf': 6, 'max_features': None}
Mejor score CV: 0.8147


Aunque en cada iteración Optuna va proponiendo combinaciones de hiperparámetros que tienden a mejorar el rendimiento del modelo, en este proyecto todavía voy ajustando manualmente los rangos y comprobando los resultados visualizando las gráficas generadas.

Soy consciente de que todo este proceso se puede automatizar por completo (ajuste dinámico de rangos, relanzar estudios según resultados previos, selección automática de hiperparámetros, etc.). Lo implementaré en futuros proyectos, pero en esta versión he preferido mantener control manual para entender mejor cómo se comporta el modelo.

Si queréis ver las gráficas, podéis descargar el programa y ejecutarlo en Colab, donde sí se renderizan porque el código corre y activa los módulos interactivos. En GitHub no aparecen porque el visor es estático. También estan subidas en el repositorio.